In [57]:
import pandas as pd
import numpy as np
import tsfresh
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import tensorflow as tf

import os

import matplotlib.pyplot as plt

# keras goodies
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, Flatten, Conv1D, Dropout, MaxPooling1D, BatchNormalization
from tensorflow.keras import optimizers
from tensorflow.keras import regularizers
from tensorflow.keras import metrics as kmetrics
import tensorflow.keras.backend as K
from tensorflow import keras

import random

In [6]:
def windows_of_sensor(sensor):
    clean_data_file = f"../pdiot-practical/pdiot-data/2021/{sensor}_recordings_clean.csv"
    base_df = pd.read_csv(clean_data_file)
    
    # not NaN
    base_df = base_df[base_df['accel_x'].notna()]
    base_df = base_df[base_df['accel_y'].notna()]
    base_df = base_df[base_df['accel_z'].notna()]
    base_df = base_df[base_df['gyro_x'].notna()]
    base_df = base_df[base_df['gyro_y'].notna()]
    base_df = base_df[base_df['gyro_z'].notna()]
    
    base_df.reset_index(drop=True, inplace=True)
    
    example_recording = base_df[base_df.activity_code == 1]
    window_size = 40 # 50 datapoints for the window size, which, at 25Hz, means 2 seconds
    step_size = 20 # this is 50% overlap

    window_number = 0 # start a counter at 0 to keep track of the window number

    large_enough_windows = [window for window in example_recording.rolling(window=window_size, min_periods=window_size) if len(window) == window_size]

    overlapping_windows = large_enough_windows[::step_size]
    for window in overlapping_windows:
        window.loc[:, 'window_id'] = window_number
        window_number += 1
    
    final_sliding_windows = pd.concat(overlapping_windows).reset_index(drop=True)
    columns_of_interest = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']
    
    feature_list = []
    for col in columns_of_interest:
        new_features = tsfresh.extract_features(timeseries_container=final_sliding_windows, column_id='window_id',
                            column_value=col, default_fc_parameters=tsfresh.feature_extraction.MinimalFCParameters())
        feature_list.append(new_features)
    feature_list = pd.concat(feature_list, axis=1)
    
    window_number = 0 # start a counter at 0 to keep track of the window number

    all_overlapping_windows = []

    for rid, group in base_df.groupby("recording_id"):
#         print(f"Processing rid = {rid}", end=' ')

        large_enough_windows = [window for window in group.rolling(window=window_size, min_periods=window_size) if len(window) == window_size]

        overlapping_windows = large_enough_windows[::step_size] 

        # then we will append a window ID to each window
        for window in overlapping_windows:
            window.loc[:, 'window_id'] = window_number
            window_number += 1

        # ??? some sind void?
        if len(overlapping_windows) == 0:
            continue

        all_overlapping_windows.append(pd.concat(overlapping_windows).reset_index(drop=True))
        
    final_sliding_windows = pd.concat(all_overlapping_windows).reset_index(drop=True)
    
    # now extract all features
    feature_list = []
    for col in columns_of_interest:
        new_features = tsfresh.extract_features(timeseries_container=final_sliding_windows, column_id='window_id',
                            column_value=col, default_fc_parameters=tsfresh.feature_extraction.MinimalFCParameters())
        feature_list.append(new_features)
    feature_list = pd.concat(feature_list, axis=1)
    
    final_sliding_windows.groupby("window_id")[['activity_type']].agg(np.min)
    
    return final_sliding_windows
    

In [7]:
final_sliding_windows_respeck = windows_of_sensor('Respeck')
final_sliding_windows_thingy = windows_of_sensor('Thingy')

/home/pustot/miniconda3/envs/pdiot/lib/python3.7/site-packages/ipykernel_launcher.py:1: DtypeWarning: Columns (11) have mixed types.Specify dtype option on import or set low_memory=False.
  """Entry point for launching an IPython kernel.
Feature Extraction: 100%|███████████████████████████████████████████████████████████████| 20/20 [00:05<00:00,  3.44it/s]
/home/pustot/miniconda3/envs/pdiot/lib/python3.7/site-packages/ipykernel_launcher.py:2: DtypeWarning: Columns (14) have mixed types.Specify dtype option on import or set low_memory=False.
  
Feature Extraction: 100%|███████████████████████████████████████████████████████████████| 20/20 [00:05<00:00,  3.61it/s]


In [52]:
class_labels = {
    'Climbing stairs': 0,
     'Descending stairs': 1,
     'Desk work': 2,
     'Falling on knees': 3,
     'Falling on the back': 4,
     'Falling on the left': 5,
     'Falling on the right': 6,
     'Lying down left': 7,
     'Lying down on back': 8,
     'Lying down on stomach': 9,
     'Lying down right': 10,
     'Movement': 11,
     'Running': 12,
     'Sitting': 13,
     'Sitting bent backward': 14,
     'Sitting bent forward': 15,
     'Standing': 16,
     'Walking at normal speed': 17
}
columns_of_interest = ['accel_x', 'accel_y', 'accel_z', 'gyro_x', 'gyro_y', 'gyro_z']

X = []
y = []

# 索引 {动作类型 -> 窗口编号}，以加快筛选随机配对窗口
thingy_act_win = final_sliding_windows_thingy[['activity_type', 'window_id']].drop_duplicates()
act_to_win = {}
for act in class_labels.keys():
    act_to_win[act] = []
for i, r in thingy_act_win.iterrows():
    act_to_win[r['activity_type']].append(r['window_id'])


for window_id, group in final_sliding_windows_respeck.groupby('window_id'):
    print(f"window_id = {window_id}", end = ' ')
    
    shape = group[columns_of_interest].values.shape
    print(f"shape = {shape}", end = ' ')
    
    act = group["activity_type"].values[0]
    times_to_concat_t_per_r = 2
    for i in range(times_to_concat_t_per_r):
        curr_id = random.choice(act_to_win[act])
        group_thingy = final_sliding_windows_thingy[final_sliding_windows_thingy['window_id'] == curr_id]
        X.append(np.concatenate((group[columns_of_interest].values, group_thingy[columns_of_interest].values), axis=1))
        y.append(class_labels[act])

window_id = 0 shape = (40, 6) window_id = 1 shape = (40, 6) window_id = 2 shape = (40, 6) window_id = 3 shape = (40, 6) window_id = 4 shape = (40, 6) window_id = 5 shape = (40, 6) window_id = 6 shape = (40, 6) window_id = 7 shape = (40, 6) window_id = 8 shape = (40, 6) window_id = 9 shape = (40, 6) window_id = 10 shape = (40, 6) window_id = 11 shape = (40, 6) window_id = 12 shape = (40, 6) window_id = 13 shape = (40, 6) window_id = 14 shape = (40, 6) window_id = 15 shape = (40, 6) window_id = 16 shape = (40, 6) window_id = 17 shape = (40, 6) window_id = 18 shape = (40, 6) window_id = 19 shape = (40, 6) window_id = 20 shape = (40, 6) window_id = 21 shape = (40, 6) window_id = 22 shape = (40, 6) window_id = 23 shape = (40, 6) window_id = 24 shape = (40, 6) window_id = 25 shape = (40, 6) window_id = 26 shape = (40, 6) window_id = 27 shape = (40, 6) window_id = 28 shape = (40, 6) window_id = 29 shape = (40, 6) window_id = 30 shape = (40, 6) window_id = 31 shape = (40, 6) window_id = 32 sha

shape = (40, 6) window_id = 262 shape = (40, 6) window_id = 263 shape = (40, 6) window_id = 264 shape = (40, 6) window_id = 265 shape = (40, 6) window_id = 266 shape = (40, 6) window_id = 267 shape = (40, 6) window_id = 268 shape = (40, 6) window_id = 269 shape = (40, 6) window_id = 270 shape = (40, 6) window_id = 271 shape = (40, 6) window_id = 272 shape = (40, 6) window_id = 273 shape = (40, 6) window_id = 274 shape = (40, 6) window_id = 275 shape = (40, 6) window_id = 276 shape = (40, 6) window_id = 277 shape = (40, 6) window_id = 278 shape = (40, 6) window_id = 279 shape = (40, 6) window_id = 280 shape = (40, 6) window_id = 281 shape = (40, 6) window_id = 282 shape = (40, 6) window_id = 283 shape = (40, 6) window_id = 284 shape = (40, 6) window_id = 285 shape = (40, 6) window_id = 286 shape = (40, 6) window_id = 287 shape = (40, 6) window_id = 288 shape = (40, 6) window_id = 289 shape = (40, 6) window_id = 290 shape = (40, 6) window_id = 291 shape = (40, 6) window_id = 292 shape = 

window_id = 527 shape = (40, 6) window_id = 528 shape = (40, 6) window_id = 529 shape = (40, 6) window_id = 530 shape = (40, 6) window_id = 531 shape = (40, 6) window_id = 532 shape = (40, 6) window_id = 533 shape = (40, 6) window_id = 534 shape = (40, 6) window_id = 535 shape = (40, 6) window_id = 536 shape = (40, 6) window_id = 537 shape = (40, 6) window_id = 538 shape = (40, 6) window_id = 539 shape = (40, 6) window_id = 540 shape = (40, 6) window_id = 541 shape = (40, 6) window_id = 542 shape = (40, 6) window_id = 543 shape = (40, 6) window_id = 544 shape = (40, 6) window_id = 545 shape = (40, 6) window_id = 546 shape = (40, 6) window_id = 547 shape = (40, 6) window_id = 548 shape = (40, 6) window_id = 549 shape = (40, 6) window_id = 550 shape = (40, 6) window_id = 551 shape = (40, 6) window_id = 552 shape = (40, 6) window_id = 553 shape = (40, 6) window_id = 554 shape = (40, 6) window_id = 555 shape = (40, 6) window_id = 556 shape = (40, 6) window_id = 557 shape = (40, 6) window_i

window_id = 820 shape = (40, 6) window_id = 821 shape = (40, 6) window_id = 822 shape = (40, 6) window_id = 823 shape = (40, 6) window_id = 824 shape = (40, 6) window_id = 825 shape = (40, 6) window_id = 826 shape = (40, 6) window_id = 827 shape = (40, 6) window_id = 828 shape = (40, 6) window_id = 829 shape = (40, 6) window_id = 830 shape = (40, 6) window_id = 831 shape = (40, 6) window_id = 832 shape = (40, 6) window_id = 833 shape = (40, 6) window_id = 834 shape = (40, 6) window_id = 835 shape = (40, 6) window_id = 836 shape = (40, 6) window_id = 837 shape = (40, 6) window_id = 838 shape = (40, 6) window_id = 839 shape = (40, 6) window_id = 840 shape = (40, 6) window_id = 841 shape = (40, 6) window_id = 842 shape = (40, 6) window_id = 843 shape = (40, 6) window_id = 844 shape = (40, 6) window_id = 845 shape = (40, 6) window_id = 846 shape = (40, 6) window_id = 847 shape = (40, 6) window_id = 848 shape = (40, 6) window_id = 849 shape = (40, 6) window_id = 850 shape = (40, 6) window_i

window_id = 1110 shape = (40, 6) window_id = 1111 shape = (40, 6) window_id = 1112 shape = (40, 6) window_id = 1113 shape = (40, 6) window_id = 1114 shape = (40, 6) window_id = 1115 shape = (40, 6) window_id = 1116 shape = (40, 6) window_id = 1117 shape = (40, 6) window_id = 1118 shape = (40, 6) window_id = 1119 shape = (40, 6) window_id = 1120 shape = (40, 6) window_id = 1121 shape = (40, 6) window_id = 1122 shape = (40, 6) window_id = 1123 shape = (40, 6) window_id = 1124 shape = (40, 6) window_id = 1125 shape = (40, 6) window_id = 1126 shape = (40, 6) window_id = 1127 shape = (40, 6) window_id = 1128 shape = (40, 6) window_id = 1129 shape = (40, 6) window_id = 1130 shape = (40, 6) window_id = 1131 shape = (40, 6) window_id = 1132 shape = (40, 6) window_id = 1133 shape = (40, 6) window_id = 1134 shape = (40, 6) window_id = 1135 shape = (40, 6) window_id = 1136 shape = (40, 6) window_id = 1137 shape = (40, 6) window_id = 1138 shape = (40, 6) window_id = 1139 shape = (40, 6) window_id 

...

window_id = 22197 shape = (40, 6) window_id = 22198 shape = (40, 6) window_id = 22199 shape = (40, 6) window_id = 22200 shape = (40, 6) window_id = 22201 shape = (40, 6) window_id = 22202 shape = (40, 6) window_id = 22203 shape = (40, 6) window_id = 22204 shape = (40, 6) window_id = 22205 shape = (40, 6) window_id = 22206 shape = (40, 6) window_id = 22207 shape = (40, 6) window_id = 22208 shape = (40, 6) window_id = 22209 shape = (40, 6) window_id = 22210 shape = (40, 6) window_id = 22211 shape = (40, 6) window_id = 22212 shape = (40, 6) window_id = 22213 shape = (40, 6) window_id = 22214 shape = (40, 6) window_id = 22215 shape = (40, 6) window_id = 22216 shape = (40, 6) window_id = 22217 shape = (40, 6) window_id = 22218 shape = (40, 6) window_id = 22219 shape = (40, 6) window_id = 22220 shape = (40, 6) window_id = 22221 shape = (40, 6) window_id = 22222 shape = (40, 6) window_id = 22223 shape = (40, 6) window_id = 22224 shape = (40, 6) window_id = 22225 shape = (40, 6) window_id = 22

window_id = 22458 shape = (40, 6) window_id = 22459 shape = (40, 6) window_id = 22460 shape = (40, 6) window_id = 22461 shape = (40, 6) window_id = 22462 shape = (40, 6) window_id = 22463 shape = (40, 6) window_id = 22464 shape = (40, 6) window_id = 22465 shape = (40, 6) window_id = 22466 shape = (40, 6) window_id = 22467 shape = (40, 6) window_id = 22468 shape = (40, 6) window_id = 22469 shape = (40, 6) window_id = 22470 shape = (40, 6) window_id = 22471 shape = (40, 6) window_id = 22472 shape = (40, 6) window_id = 22473 shape = (40, 6) window_id = 22474 shape = (40, 6) window_id = 22475 shape = (40, 6) window_id = 22476 shape = (40, 6) window_id = 22477 shape = (40, 6) window_id = 22478 shape = (40, 6) window_id = 22479 shape = (40, 6) window_id = 22480 shape = (40, 6) window_id = 22481 shape = (40, 6) window_id = 22482 shape = (40, 6) window_id = 22483 shape = (40, 6) window_id = 22484 shape = (40, 6) window_id = 22485 shape = (40, 6) window_id = 22486 shape = (40, 6) window_id = 22

window_id = 22729 shape = (40, 6) window_id = 22730 shape = (40, 6) window_id = 22731 shape = (40, 6) window_id = 22732 shape = (40, 6) window_id = 22733 shape = (40, 6) window_id = 22734 shape = (40, 6) window_id = 22735 shape = (40, 6) window_id = 22736 shape = (40, 6) window_id = 22737 shape = (40, 6) window_id = 22738 shape = (40, 6) window_id = 22739 shape = (40, 6) window_id = 22740 shape = (40, 6) window_id = 22741 shape = (40, 6) window_id = 22742 shape = (40, 6) window_id = 22743 shape = (40, 6) window_id = 22744 shape = (40, 6) window_id = 22745 shape = (40, 6) window_id = 22746 shape = (40, 6) window_id = 22747 shape = (40, 6) window_id = 22748 shape = (40, 6) window_id = 22749 shape = (40, 6) window_id = 22750 shape = (40, 6) window_id = 22751 shape = (40, 6) window_id = 22752 shape = (40, 6) window_id = 22753 shape = (40, 6) window_id = 22754 shape = (40, 6) window_id = 22755 shape = (40, 6) window_id = 22756 shape = (40, 6) window_id = 22757 shape = (40, 6) window_id = 22

window_id = 22990 shape = (40, 6) window_id = 22991 shape = (40, 6) window_id = 22992 shape = (40, 6) window_id = 22993 shape = (40, 6) window_id = 22994 shape = (40, 6) window_id = 22995 shape = (40, 6) window_id = 22996 shape = (40, 6) window_id = 22997 shape = (40, 6) window_id = 22998 shape = (40, 6) window_id = 22999 shape = (40, 6) window_id = 23000 shape = (40, 6) window_id = 23001 shape = (40, 6) window_id = 23002 shape = (40, 6) window_id = 23003 shape = (40, 6) window_id = 23004 shape = (40, 6) window_id = 23005 shape = (40, 6) window_id = 23006 shape = (40, 6) window_id = 23007 shape = (40, 6) window_id = 23008 shape = (40, 6) window_id = 23009 shape = (40, 6) window_id = 23010 shape = (40, 6) window_id = 23011 shape = (40, 6) window_id = 23012 shape = (40, 6) window_id = 23013 shape = (40, 6) window_id = 23014 shape = (40, 6) window_id = 23015 shape = (40, 6) window_id = 23016 shape = (40, 6) window_id = 23017 shape = (40, 6) window_id = 23018 shape = (40, 6) window_id = 23

window_id = 23265 shape = (40, 6) window_id = 23266 shape = (40, 6) window_id = 23267 shape = (40, 6) window_id = 23268 shape = (40, 6) window_id = 23269 shape = (40, 6) window_id = 23270 shape = (40, 6) window_id = 23271 shape = (40, 6) window_id = 23272 shape = (40, 6) window_id = 23273 shape = (40, 6) window_id = 23274 shape = (40, 6) window_id = 23275 shape = (40, 6) window_id = 23276 shape = (40, 6) window_id = 23277 shape = (40, 6) window_id = 23278 shape = (40, 6) window_id = 23279 shape = (40, 6) window_id = 23280 shape = (40, 6) window_id = 23281 shape = (40, 6) window_id = 23282 shape = (40, 6) window_id = 23283 shape = (40, 6) window_id = 23284 shape = (40, 6) window_id = 23285 shape = (40, 6) window_id = 23286 shape = (40, 6) window_id = 23287 shape = (40, 6) window_id = 23288 shape = (40, 6) window_id = 23289 shape = (40, 6) window_id = 23290 shape = (40, 6) window_id = 23291 shape = (40, 6) window_id = 23292 shape = (40, 6) window_id = 23293 shape = (40, 6) window_id = 23

In [53]:
X = np.asarray(X)
y = np.asarray(y)

print(f"X shape = {X.shape}")
print(f"y shape = {y.shape}")

# np.save('X.npy', X)
# np.save('y.npy', y)

# X = np.load('X.npy')

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2, train_size=0.8)

y_train = np.asarray(pd.get_dummies(y_train), dtype=np.float32)
y_test = np.asarray(pd.get_dummies(y_test), dtype=np.float32)

print(f"X_train shape = {X_train.shape}")
print(f"y_train shape = {y_train.shape}")

print(f"X_test shape = {X_test.shape}")
print(f"y_test shape = {y_test.shape}")



X shape = (46734, 40, 12)
y shape = (46734,)
X_train shape = (37387, 40, 12)
y_train shape = (37387, 18)
X_test shape = (9347, 40, 12)
y_test shape = (9347, 18)


In [58]:
window_size = 40

filters = 128
kernel_size = 2
n_features = 12
activation='relu'
n_classes = 18



model = Sequential()

model.add(Conv1D(filters=16, kernel_size=kernel_size, activation='relu', 
                 input_shape=(window_size, n_features)))
# model.add(BatchNormalization())
# model.add(Activation(activation))

model.add(Conv1D(filters=32, kernel_size=kernel_size, activation='relu'))
model.add(BatchNormalization())
# model.add(Activation(activation))
model.add(MaxPooling1D(pool_size=2, strides=2))

model.add(Conv1D(filters=64, kernel_size=kernel_size, activation='relu'))
# model.add(BatchNormalization())
# model.add(Activation(activation))
model.add(MaxPooling1D(pool_size=2, strides=2))

model.add(Conv1D(filters=128, kernel_size=kernel_size, activation='relu'))
model.add(BatchNormalization())
# model.add(Activation(activation))
model.add(MaxPooling1D(pool_size=2, strides=2))

model.add(Flatten())
model.add(Dense(100, activation='relu'))
model.add(Dense(n_classes, activation='softmax'))

model.summary()



model.compile(
    optimizer=optimizers.SGD(lr=0.03),
    loss='binary_crossentropy',
    metrics = ['accuracy'])


model.fit(X_train, y_train,
        batch_size=8, epochs=15)


# stats
y_pred_ohe = model.predict(X_test)
y_pred_labels = np.argmax(y_pred_ohe, axis=1)
y_true_labels = np.argmax(y_test, axis=1)
print("*" * 80)
print("Classification report")
print("*" * 80)
print(classification_report(y_true_labels, y_pred_labels))

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_8 (Conv1D)            (None, 39, 16)            400       
_________________________________________________________________
conv1d_9 (Conv1D)            (None, 38, 32)            1056      
_________________________________________________________________
batch_normalization_4 (Batch (None, 38, 32)            128       
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 19, 32)            0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, 18, 64)            4160      
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 9, 64)             0         
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 8, 128)           

In [59]:
# Convert the model. （这个准确率还好低……下一步多研究一下模型）
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model. （注意我默认把模型先生成到当前目录了，还需要手动添加到工程里）
with open('model.tflite', 'wb') as f:
    f.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpu52ta302/assets


INFO:tensorflow:Assets written to: /tmp/tmpu52ta302/assets
2021-11-09 01:11:56.599653: I tensorflow/core/grappler/devices.cc:78] Number of eligible GPUs (core count >= 8, compute capability >= 0.0): 0 (Note: TensorFlow was not compiled with CUDA or ROCm support)
2021-11-09 01:11:56.600190: I tensorflow/core/grappler/clusters/single_machine.cc:356] Starting new session
2021-11-09 01:11:56.604364: I tensorflow/core/grappler/optimizers/meta_optimizer.cc:928] Optimization results for grappler item: graph_to_optimize
  function_optimizer: function_optimizer did nothing. time = 0.002ms.
  function_optimizer: function_optimizer did nothing. time = 0ms.

2021-11-09 01:11:56.726205: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:316] Ignored output_format.
2021-11-09 01:11:56.726431: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:319] Ignored drop_control_dependency.


2021 Clean 数据集：

kernel_size|Optmz|cnt_Cov1D|lr|batch_size|epochs|acc-train|acc-test
-|-|-|-|-|-|-|-
2   |  SGD|   4|  0.03|   8|  10|  0.83|  0.83
2   |  SGD|   4|  0.03|   8|  15|  0.85|  0.85